In [ ]:
import xarray as xr
import earthaccess
import boto3
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import warnings
from IPython.display import display, Markdown

In [ ]:
# Authenticate using Earthdata Login prerequisite files
auth = earthaccess.login()

# Search for the granule by DOI
results = earthaccess.search_data(
    doi='10.5067/RKPHT8KC1Y1T',
    temporal=("2022-03-01", "2022-03-31"),
)

In [ ]:
results

In [ ]:
fn = earthaccess.open(results)

In [ ]:
ds = xr.open_mfdataset(fn)

In [ ]:
ds['SNODP']

In [ ]:
ds['SNODP'][0,:,:].plot(vmin=0, vmax=1)

In [ ]:
mask_lon = (ds.lon >= -168.75) & (ds.lon <= -136.01)
mask_lat = (ds.lat >= 52.64) & (ds.lat <= 71.59)

ds_ak = ds.where(mask_lon & mask_lat, drop=True)

In [ ]:
ds_ak['SNODP'][0,:,:].plot(vmin=0, vmax=1.25)

In [ ]:
ak_daily_mean = ds_ak.resample(time='D').mean()

In [ ]:
ak_daily_mean

In [ ]:
ak_daily_mean['SNODP'][:,23,34].plot() 